In [ ]:
from jupyter_dash import JupyterDash     # need Dash version 1.21.0 or higher
from dash.dependencies import Input, Output, State
import dash_table
import dash_core_components as dcc
import dash_html_components as html

import pandas as pd
import plotly.express as px
import pymongo
from pymongo import MongoClient
import getpass
import json
from datetime import datetime


#-----------------------------------------------Connection to Mongo------------------------------------------------------------------------
# Connect to local server
password= getpass.getpass()
username='monia'
client = MongoClient("mongodb+srv://{}:{}@cluster0.wr80t.mongodb.net/myFirstDatabase?retryWrites=true&w=majority".format(username, password))

# Create database called animals
mydb = client["sentiment"]
# Create Collection (table) called shelterA
collection = mydb.df_sentiment

In [ ]:
#Inserting elements into the db
df=pd.read_csv('df_by_day.csv')
df=df.to_json(orient='records')
parsed = json.loads(df)
collection.insert_many(parsed)

In [69]:
#--------------------------------------------------Beginning Dash-------------------------------------------------------------------------
app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

app.layout = html.Div([
    html.Div(id='mongo-datatable', children=[]),
    # activated once/week or when page refreshed
    dcc.Interval(id='interval_db', interval=86400000 * 7, n_intervals=0), 
    html.Div(id="show-graphs", children=[]),
])

# Display Datatable with data from Mongo database 
@app.callback(Output('mongo-datatable', 'children'),
              [Input('interval_db', 'n_intervals')])
def populate_datatable(n_intervals):
    print(n_intervals)
    # Convert the Collection (table) date to a pandas DataFrame
    df = pd.DataFrame(list(collection.find()))
    #Drop the _id column generated automatically by Mongo
    df = df.iloc[:, 1:]

    return [
        dash_table.DataTable(
            id='my-table',
            columns=[{
                'name': x,
                'id': x,
            } for x in df.columns],
            data=df.to_dict('records'),
            editable=True,
            row_deletable=True,
            filter_action="native",
            filter_options={"case": "sensitive"},
            sort_action="native",  # give user capability to sort columns
            sort_mode="single",  # sort across 'multi' or 'single' columns
            page_current=0,  # page number that user is on
            page_size=6,  # number of rows visible per page
            style_cell={'textAlign': 'left', 'minWidth': '100px',
                        'width': '100px', 'maxWidth': '100px'},
        )
    ]



# Create graphs from DataTable data ***************************************
@app.callback(
    Output('show-graphs', 'children'),
    Input('my-table', 'data')
)
def add_row(data):
    df_graph = pd.DataFrame(data) 
    df_graph['date']=pd.to_datetime(df_graph['date'], format='%d/%m/%Y') #turn date into date_time type
    df_graph= df_graph.sort_values('date') #sort values by date
    mapping={'Negative':-1, 'Positive':1, 'Neutral':0}
    df_graph['label']= df_graph['label'].map(mapping) #map strings to values (-1.0,1)
    df_graph['overall_sentiment']= df_graph['count']*df_graph['label'] #sum the values
    df= df_graph.groupby(by='date', as_index=False).sum('overall_sentiment')
    
    fig_hist1 = px.histogram(df_graph, x='label',color="label")
    fig_hist2 = px.line(df, x="date", y='overall_sentiment')
    return [
        html.Div(children=[dcc.Graph(figure=fig_hist1)], className="six columns"),
        html.Div(children=[dcc.Graph(figure=fig_hist2)], className="six columns")
    ]


app.run_server(mode='inline')

0
